In [1]:
%load_ext autoreload
%autoreload 2

# Load model

In [2]:
from pointnet2.models.pointnet2_msg_sem import Pointnet2MSG
import torch.nn as nn

model = Pointnet2MSG(num_classes=13, input_channels=0, use_xyz=True)
model.cuda()

Pointnet2MSG(
  (SA_modules): ModuleList(
    (0): PointnetSAModuleMSG(
      (groupers): ModuleList(
        (0): QueryAndGroup()
        (1): QueryAndGroup()
      )
      (mlps): ModuleList(
        (0): SharedMLP(
          (layer0): Conv2d(
            (conv): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer1): Conv2d(
            (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer2): Conv2d(
            (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
       

In [3]:
import etw_pytorch_utils.pytorch_utils as pt_utils
import torch

checkpoint_name = "checkpoints/poitnet2_semseg_best.pth.tar"
checkpoint = torch.load(checkpoint_name)
model.load_state_dict(checkpoint["model_state"])

# Load dataset 

In [4]:
from pointnet2.data.Indoor3DSemSegLoader import Indoor3DSemSeg
num_points = 4096
train_set = Indoor3DSemSeg(num_points, download=False)

In [5]:
# num of 3d models, num of points in each model, xyz cood
print(train_set.points.shape)
print(train_set.labels.shape)

(16733, 4096, 3)
(16733, 4096)


In [6]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure()
ax = Axes3D(fig)

data_to_draw = train_set.points[12]

sequence_containing_x_vals = data_to_draw[:, 0]
sequence_containing_y_vals = data_to_draw[:, 1]
sequence_containing_z_vals = data_to_draw[:, 2]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [7]:
from torch.utils.data import DataLoader
batch_size = 1

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=8,
    shuffle=True,
)

In [8]:
import torch.nn as nn
from pointnet2.models.pointnet2_msg_sem import model_fn_decorator

model_fn = model_fn_decorator(nn.CrossEntropyLoss())

In [9]:
model.eval()
model.cuda()
for data in train_loader:
    print(data)
    _, loss, eval_res, features = model_fn(model, data, eval=True)
    break
print(features.shape)

[tensor([[[-0.0600, -0.4690,  1.5780],
         [ 0.4630, -0.4970,  1.9220],
         [-0.0300, -0.4550,  0.1680],
         ...,
         [-0.4430,  0.2570,  0.0640],
         [-0.4700,  0.1890,  2.7140],
         [ 0.3910, -0.2530,  0.0590]]]), tensor([[6, 6, 6,  ..., 1, 0, 1]])]
torch.Size([1, 128, 4096])


In [10]:
import numpy as np
B = 1
N = 125
cube = torch.empty(B, N, 3).cuda()
labels = torch.from_numpy(np.zeros((1, B * N), dtype = int)).view(B, N).cuda()

for x in range(0, 5, 1):
    for y in range(0, 5, 1):
        for z in range(0, 5, 1):
            cube[0, x + y*5 + z*25, :] = torch.tensor([x,y,z])

# cube = cube.unsqueeze(-1)
            
cube1_xyz = cube - 5
cube2_xyz = cube + torch.rand_like(cube)

In [11]:
print(cube1_xyz[0, 0, :])
print(labels.shape)

tensor([-5., -5., -5.], device='cuda:0')
torch.Size([1, 125])


In [12]:
cube1_label_tor = labels
cube2_label_tor = labels

cube1_data_tor = (cube1_xyz, cube1_label_tor)
cube2_data_tor = (cube2_xyz, cube2_label_tor)

In [13]:
_, _, _, f1 = model_fn(model, cube1_data_tor, eval=True)
_, _, _, f2 = model_fn(model, cube2_data_tor, eval=True)
print(f1.shape)
print(f1)
print(f2)

torch.Size([1, 128, 125])
tensor([[[3.1410, 3.1410, 3.1410,  ..., 3.1410, 3.1410, 3.1410],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.7085, 0.7085, 0.7085,  ..., 0.7085, 0.7085, 0.7085],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       device='cuda:0')
tensor([[[3.1410, 3.1410, 3.1412,  ..., 3.1410, 3.1410, 3.1410],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.7085, 0.7086, 0.7088,  ..., 0.7085, 0.7085, 0.7085],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       device='cuda:0')


In [14]:
def compute_vec_dist(f1, f2):
    return np.sqrt(np.sum(np.power((f1 - f2).cpu().numpy(), 2.0)))

In [15]:
dist = compute_vec_dist(f1, f2)
print(dist)

2.3220127
